In [4]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

sk


In [5]:
import re
import os, json

from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

In [6]:
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool

# Tool 정의 
@tool
def tavily_search_func(query: str) -> str:
    """Searches the internet for information that does not exist in the database or for the latest information."""

    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)

    formatted_docs = "\n---\n".join([
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
        ])

    if len(formatted_docs) > 0:
        return formatted_docs
    
    return "관련 정보를 찾을 수 없습니다."

In [7]:
from langchain_openai import ChatOpenAI

# LLM 모델 
#llm = ChatOpenAI(model="gpt-4o-mini")
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
print(llm.model_name)

meta-llama/llama-4-scout-17b-16e-instruct


In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_community.document_loaders import WikipediaLoader
from pydantic import BaseModel, Field

# WikipediaLoader를 사용하여 위키피디아 문서를 검색하고 텍스트로 반환하는 함수 
def wiki_search_and_summarize(input_data: dict):
    wiki_loader = WikipediaLoader(query=input_data["query"], load_max_docs=2, lang="ko")
    wiki_docs = wiki_loader.load()

    formatted_docs =[
        f'<Document source="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
        for doc in wiki_docs
        ]
    
    return formatted_docs

class WikiSummarySchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")


# 요약 프롬프트 템플릿
summary_prompt = ChatPromptTemplate.from_template(
    "Summarize the following text in a concise manner:\n\n{context}\n\nSummary:"
)

# LLM 및 요약 체인 설정
summary_chain = (
    {"context": RunnableLambda(wiki_search_and_summarize)}
    | summary_prompt | llm
)

wiki_summary = summary_chain.as_tool(
    name="wiki_summary",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a summarized text. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSummarySchema
)


C:\Users\user\AppData\Local\Temp\ipykernel_25916\1564754013.py:34: LangChainBetaWarning: This API is in beta and may change in the future.
  wiki_summary = summary_chain.as_tool(


In [9]:
from langchain.document_loaders import TextLoader
from langchain_core.documents import Document

# 메뉴판 텍스트 데이터를 로드
loader = TextLoader("./data/restaurant_menu.txt", encoding="utf-8")
documents = loader.load()


# 문서 분할 (Chunking)
def split_menu_items(document):
    """
    메뉴 항목을 분리하는 함수 
    """
    # 정규표현식 정의 
    pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
    menu_items = re.findall(pattern, document.page_content, re.DOTALL)
    
    # 각 메뉴 항목을 Document 객체로 변환
    menu_documents = []
    for i, item in enumerate(menu_items, 1):
        # 메뉴 이름 추출
        menu_name = item.split('\n')[0].split('.', 1)[1].strip()
        
        # 새로운 Document 객체 생성
        menu_doc = Document(
            page_content=item.strip(),
            metadata={
                "source": document.metadata['source'],
                "menu_number": i,
                "menu_name": menu_name
            }
        )
        menu_documents.append(menu_doc)
    
    return menu_documents


# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)


In [10]:

from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings

embeddings_model = OllamaEmbeddings(model="qwen2.5:1.5b") 

# FAISS 인덱스 생성
menu_db = FAISS.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model
)

# FAISS 인덱스 저장 (선택사항)
menu_db.save_local("./db/menu_db")


# Retriever 생성
menu_retriever = menu_db.as_retriever(
    search_kwargs={'k': 2},
)

# 쿼리 테스트
query = "시그니처 스테이크의 가격과 특징은 무엇인가요?"
docs = menu_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()


검색 결과: 2개
메뉴 번호: 3
메뉴 이름: 연어 타르타르

메뉴 번호: 1
메뉴 이름: 시그니처 스테이크



In [11]:
# 와인 메뉴 텍스트 데이터를 로드
loader = TextLoader("./data/restaurant_wine.txt", encoding="utf-8")
documents = loader.load()

# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")


# FAISS 인덱스 생성
wine_db = FAISS.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model
)

wine_db.save_local("./db/wine_db")

wine_retriever = wine_db.as_retriever(
    search_kwargs={'k': 2},
)

query = "스테이크와 어울리는 와인을 추천해주세요."
docs = wine_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

총 10개의 메뉴 항목이 처리되었습니다.

메뉴 번호: 1
메뉴 이름: 샤토 마고 2015
내용:
1. 샤토 마고 2015
   • 가격: ₩450,000
   • 주요 품종: 카베르네 소비뇽, 메를로, 카베르네 프랑, 쁘띠 베르도
   • 설명: 보르도 메독 지역의 프리미엄 ...

메뉴 번호: 2
메뉴 이름: 돔 페리뇽 2012
내용:
2. 돔 페리뇽 2012
   • 가격: ₩380,000
   • 주요 품종: 샤르도네, 피노 누아
   • 설명: 프랑스 샴페인의 대명사로 알려진 프레스티지 큐베입니다. 시트러스...
검색 결과: 2개
메뉴 번호: 10
메뉴 이름: 그랜지 2016

메뉴 번호: 6
메뉴 이름: 바롤로 몬프리바토 2017



In [12]:
from langchain_core.tools import tool
from typing import List
from langchain_core.documents import Document

# menu db 벡터 저장소 로드
menu_db = FAISS.load_local(
    "./db/menu_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

@tool
def db_search_menu_func(query: str) -> List[Document]:
    """
    Securely retrieve and access authorized restaurant menu information from the encrypted database.
    Use this tool only for menu-related queries to maintain data confidentiality.
    """
    docs = menu_db.similarity_search(query, k=2)
    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 메뉴 정보를 찾을 수 없습니다.")]

# wine db 벡터 저장소 로드
wine_db = FAISS.load_local(
    "./db/wine_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

@tool
def db_search_wine_func(query: str) -> List[Document]:
   """
   Securely retrieve and access authorized restaurant wine information from the encrypted database.
   Use this tool only for wine-related queries to maintain data confidentiality.
   """
   docs = wine_db.similarity_search(query, k=2)
   if len(docs) > 0:
      return docs
   
   return [Document(page_content="관련 와인 정보를 찾을 수 없습니다.")]


In [14]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# 4개의 검색 도구를 LLM에 바인딩
tools = [tavily_search_func, wiki_summary, db_search_menu_func, db_search_wine_func]
llm_with_tools = llm.bind_tools(tools=tools)

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def restaurant_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_menu_func":
            tool_message = db_search_menu_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_wine_func":
            tool_message = db_search_wine_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)            

    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
response = restaurant_menu_chain.invoke("시그니처 스테이크의 가격과 특징은 무엇인가요? 와인도 추천 해주세요.")

# 응답 출력 
print(response.content)

AuthenticationError: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}

In [ ]:
# 체인 실행
response = restaurant_menu_chain.invoke("파스타의 역사 또는 유래를 알려주세요. 서울 강남의 파스타 맛집을 추천해주세요.")

# 응답 출력 
print(response.content)

wiki_summary: 
{'name': 'wiki_summary', 'args': {'query': 'pasta history'}, 'id': 't95xwzd8h', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tavily_search_func: 
{'name': 'tavily_search_func', 'args': {'query': 'Seoul Gangnam pasta restaurant recommendation'}, 'id': 'h7ntcmcxw', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content='content=\'The text appears to be about pasta and a specific Italian dish called "Amatriciana". Here is a concise summary:\\n\\n**Pasta**\\n\\n* Pasta is a traditional Italian food made from durum wheat semolina or flour and eggs.\\n* There are many types of pasta, including dried pasta (pasta secca) and fresh pasta (pasta fresca).\\n* Pasta can be shaped in various forms, such as long, short, tubular, flat, and stuffed.\\n\\n**Amatriciana**\\n\\n* Amatriciana is a pasta sauce origin